### Case Study 2: GIS Map Visualization

**Objective:** Visualize the speed and elevation on a map. Geographic Information Systems (GIS) are an important technology to view spatially distributed data. The GIS maps can help to identify factors related to location.

Machine Learning for Engineers: [Automotive Monitoring](https://www.apmonitor.com/pds/index.php/Main/AutomotiveMonitoring)
- Description: Machine learning project with automotive data. Data includes travel distance, time, fuel rate, air flow, oxygen ratio, and other parameters available from an OBD2 interface.
- [Course Overview](https://apmonitor.com/pds)
- [Course Schedule](https://apmonitor.com/pds/index.php/Main/CourseSchedule)

<img width=400px align=left src='https://apmonitor.com/pds/uploads/Main/automotive_monitoring.png'>

### Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Import Data and View Columns

Import data, set time index, and print data columns

In [ ]:
url = 'http://apmonitor.com/pds/uploads/Main/auto_trip.zip'
data = pd.read_csv(url)
data = data[::100]

# set time index
data['time'] = pd.to_datetime(data['time'])
data = data.set_index('time')

# print data columns
for x in data.columns:
    print(x)

In [ ]:
data.sample(5)

### Select Columns of Interest

In [ ]:
# fill in NaNs - forward fill
data.fillna(method='ffill',inplace=True)
# fill in NaNs - backward fill
data.fillna(method='bfill',inplace=True)
dr = len(data)
data.describe()

### Plot Data

In [ ]:
data.plot(subplots=True,figsize=(10,30))
plt.show()

### View GPS Points on Map

Create function to view GPS Data on Map from [Open Street Map](https://osm.org/go/TxdH5).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import requests
from PIL import Image
from io import BytesIO

def deg2num(lat_deg, lon_deg, zoom):
  lat_rad = math.radians(lat_deg)
  n = 2.0 ** zoom
  xtile = int((lon_deg + 180.0) / 360.0 * n)
  ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
  return (xtile, ytile)
  
def num2deg(xtile, ytile, zoom):
  n = 2.0 ** zoom
  lon_deg = xtile / n * 360.0 - 180.0
  lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
  lat_deg = math.degrees(lat_rad)
  return (lat_deg, lon_deg)
  
def getImageCluster(lat_deg, lon_deg, delta_lat,  delta_long):
    smurl = r"http://a.tile.openstreetmap.org/{0}/{1}/{2}.png"

    # find the correct zoom level
    zoom = 1; nt = 1
    while nt<3 and zoom<=13:
        zoom+=1 # increment zoom level
        xmin, ymax =deg2num(lat_deg, lon_deg, zoom)
        xmax, ymin =deg2num(lat_deg + delta_lat, lon_deg + delta_long, zoom)
        nt = (xmax-xmin)*(ymax-ymin)
    print('Number of tiles: ',nt)
    
    # calculate bounding box for all tiles
    lat1,long1 = num2deg(xmin,ymin,zoom)
    lat2,long2 = num2deg(xmax,ymax,zoom)
    nlong = xmax-xmin
    nlat  = ymax-ymin
    w = (nlong)*256
    h = (nlat)*256
    print('Tiles: ',nlat,nlong)
    bb = [w,h,lat1,lat2,long1,long2]
    
    Cluster = Image.new('RGB',((xmax-xmin+1)*256-1,(ymax-ymin+1)*256-1) ) 
    for xtile in range(xmin, xmax+1):
        for ytile in range(ymin, ymax+1):
            try:
                imgurl=smurl.format(zoom, xtile, ytile)
                imgstr = requests.get(imgurl).content
                tile = Image.open(BytesIO(imgstr))
                Cluster.paste(tile, box=((xtile-xmin)*256 ,  (ytile-ymin)*255))
            except: 
                print("Couldn't download image")
                tile = None
    return [bb,Cluster]    

### Display GPS Route on Static Map

See the [Open Street Map (OSM) Usage Policy](https://operations.osmfoundation.org/policies/tiles/) for policies that prevent overloading the public server.

In [ ]:
x = data['Longitude'].values
y = data['Latitude'].values
lat1 = y.max(); lat2 = y.min()
long1 = x.min(); long2 = x.max()
dx = long2-long1
dy = lat1-lat2

bb,a = getImageCluster(lat2, long1, dy,  dx)
fig = plt.figure(figsize=(10,8))
fig.patch.set_facecolor('white')
plt.imshow(np.asarray(a))

# plot scaled data on map
w,h,bblat1,bblat2,bblong1,bblong2 = bb
xs = (x-bblong1) * w/(bblong2-bblong1)
ys = (y-bblat1) * h/(bblat2-bblat1)
plt.plot(xs,ys,'r-')

plt.axis('off')
plt.savefig('map_GPS.png')
plt.show()

### Display GPS with Plotly Express (Less Complicated, Fewer Options)

In [ ]:
import plotly.express as px

df = px.data.carshare()
fig = px.scatter_mapbox(data, lat="Latitude", lon="Longitude", \
                        color="Vehicle speed (mph)", size="Altitude (GPS) (feet)", \
                        color_continuous_scale=px.colors.cyclical.IceFire, size_max=5, zoom=6)
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)
fig.show()

### GPS Route with Plotly (More Complicated, More Options)

In [ ]:
import plotly.graph_objects as go
import numpy as np

# minimum and maximum latitudes for plot
min_lat = data['Latitude'].min()
max_lat = data['Latitude'].max()
d_lat = abs(max_lat - min_lat)

# minimum and maximum longitudes for plot
min_lon = data['Longitude'].min()
max_lon = data['Longitude'].max()
d_lon = abs(max_lon - min_lon)

# Scattergeo or Scattermapbox
fig = go.Figure(data=go.Scattermapbox(
    lat = data['Latitude'],
    lon = data['Longitude'],
    text = data['Altitude (GPS) (feet)'].astype(int).astype(str) + ' ft ' + 
            data['Speed (GPS) (mph)'].astype(int).astype(str) + ' mph',
    marker = dict(
        color = data['Speed (GPS) (mph)'],
        size = data['Altitude (GPS) (feet)'].values/1000.0,
        colorscale = 'sunset',
        reversescale = False,
        opacity = 0.7,
        colorbar = dict(
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            showticksuffix = "last",
            dtick = 10
        )
    )
))

fig.update_layout(
    title='Vehicle Data',
    mapbox = dict(style="stamen-terrain",
                  bearing=0,
                  center=go.layout.mapbox.Center(
                    lat=np.mean([min_lat,max_lat]),
                    lon=np.mean([min_lon,max_lon])),
                  pitch=0,zoom=5),
)
fig.show()